Preprocessing and feature engineering

In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install plotly_express
!pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


  Using cached plotly_express-0.4.1-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached plotly_express-0.4.1-py2.py3-none-any.whl (2.9 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Prem Shah\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load the dataset
df1 = pd.read_csv("smmh.csv")

In [ ]:
df1.shape

(481, 21)

In [ ]:
# Make a working copy
df = df1.copy()

In [ ]:
# Drop rows with missing values
df_clean = df.dropna()

In [ ]:
#print columns: remove timestamp not important

In [ ]:
# Drop irrelevant columns (like Timestamp)
df_clean = df_clean.drop(columns=['Timestamp'])

In [ ]:
# check the value counts in the Gender column using this line of code:
df_clean['2. Gender'].value_counts()

2. Gender
Female                 245
Male                   201
Nonbinary                1
Non-binary               1
unsure                   1
Non binary               1
There are others???      1
Name: count, dtype: int64

Since only "Male" and "Female" have a meaningful count, and the rest (like "Nonbinary", "unsure", etc.) appear only once, you can simplify this column into two main groups:

In [ ]:
# Keep only Male and Female, drop others
df_clean = df_clean[df_clean['2. Gender'].isin(['Male', 'Female'])]
print(df_clean['2. Gender'].value_counts())

2. Gender
Female    245
Male      201
Name: count, dtype: int64


In [ ]:
# Encode as binary: Male = 1, Female = 0
df_clean['Gender_binary'] = df_clean['2. Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
df_clean = df_clean.drop(columns=['2. Gender'])

In [ ]:
df_clean['3. Relationship Status'].value_counts()

3. Relationship Status
Single               261
Married               94
In a relationship     85
Divorced               6
Name: count, dtype: int64

In [ ]:
#function to do one hot coding

In [ ]:
# Step 1: Define allowed categories
allowed_relationships = ['Single', 'Married', 'In a relationship']

# Step 2: Filter rows where 'Relationship Status' is one of the allowed values
df_clean = df_clean[df_clean['3. Relationship Status'].isin(allowed_relationships)]

In [ ]:
# Apply one-hot encoding to '3. Relationship Status'
df_clean = pd.get_dummies(df_clean, columns=['3. Relationship Status'], drop_first=True)

In [ ]:
# Convert True/False to 1/0 for relationship status columns
df_clean[['3. Relationship Status_Married', '3. Relationship Status_Single']] = \
    df_clean[['3. Relationship Status_Married', '3. Relationship Status_Single']].astype(int)


In [ ]:
df_clean['4. Occupation Status'].value_counts()

4. Occupation Status
University Student    277
Salaried Worker       118
School Student         42
Retired                 3
Name: count, dtype: int64

dropping Retired due to low count

In [ ]:
# Drop "Retired" from Occupation
# Filter out rows where occupation is 'Retired'
df_clean = df_clean[df_clean['4. Occupation Status'] != 'Retired']

In [ ]:
# Apply one-hot encoding
df_clean = pd.get_dummies(df_clean, columns=['4. Occupation Status'], drop_first=True)

In [ ]:
# Convert boolean True/False to integer 1/0 for occupation columns
occupation_cols = [col for col in df_clean.columns if '4. Occupation Status_' in col]
df_clean[occupation_cols] = df_clean[occupation_cols].astype(int)

In [ ]:
df_clean['5. What type of organizations are you affiliated with?'].value_counts()

5. What type of organizations are you affiliated with?
University                        236
Private                            54
School                             41
Company                            29
University, Company                19
University, Private                16
School, University                  9
Company, Private                    7
Goverment                           5
University, Company, Private        5
School, University, Private         4
University, N/A                     3
University, Goverment               2
School, Company                     2
School, N/A                         2
University, Goverment, Private      1
School, Private                     1
University, Company, Goverment      1
Name: count, dtype: int64

In [ ]:
# Clean whitespace and split multiple affiliations
df_clean['5. What type of organizations are you affiliated with?'] = \
    df_clean['5. What type of organizations are you affiliated with?'].fillna('').apply(lambda x: [i.strip() for i in x.split(',')])

# Use MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the multi-label column
affiliation_dummies = pd.DataFrame(
    mlb.fit_transform(df_clean['5. What type of organizations are you affiliated with?']),
    columns=['Aff_' + col for col in mlb.classes_],
    index=df_clean.index
)

# Concatenate with original DataFrame
df_clean = pd.concat([df_clean.drop(columns=['5. What type of organizations are you affiliated with?']), affiliation_dummies], axis=1)


In [ ]:
# Check unique values
df_clean['6. Do you use social media?'].value_counts()

6. Do you use social media?
Yes    435
No       2
Name: count, dtype: int64

In [ ]:
df_clean['6. Do you use social media?'] = df_clean['6. Do you use social media?'].map({'Yes': 1, 'No': 0})

In [ ]:
df_clean['7. What social media platforms do you commonly use?'].value_counts()

7. What social media platforms do you commonly use?
Facebook, Instagram, YouTube                                                 31
Facebook, YouTube                                                            27
Facebook, Instagram, YouTube, Snapchat                                       24
Facebook, Instagram, YouTube, Discord                                        18
Facebook                                                                     17
                                                                             ..
Facebook, Discord, Reddit                                                     1
Instagram, Discord, Reddit, Pinterest, TikTok                                 1
Twitter, Instagram, YouTube, Snapchat, Discord, Reddit, Pinterest, TikTok     1
Facebook, YouTube, Snapchat                                                   1
Instagram, YouTube, Snapchat, Discord, Reddit, Pinterest, TikTok              1
Name: count, Length: 121, dtype: int64

In [ ]:
# Create dummy variables for top platforms
platforms = ['Instagram', 'Facebook', 'Twitter', 'Snapchat', 'LinkedIn', 'YouTube']

for platform in platforms:
    df_clean[f'platform_{platform}'] = df_clean['7. What social media platforms do you commonly use?']\
                                        .str.contains(platform, case=False, na=False).astype(int)


In [ ]:
#Drop the original column (optional)
df_clean.drop(columns=['7. What social media platforms do you commonly use?'], inplace=True)


In [ ]:
# Check unique values
df_clean['8. What is the average time you spend on social media every day?'].value_counts()

8. What is the average time you spend on social media every day?
More than 5 hours        107
Between 2 and 3 hours     92
Between 3 and 4 hours     85
Between 1 and 2 hours     62
Between 4 and 5 hours     61
Less than an Hour         30
Name: count, dtype: int64

In [ ]:
time_mapping = {
    'Less than an Hour': 0.5,
    'Between 1 and 2 hours': 1.5,
    'Between 2 and 3 hours': 2.5,
    'Between 3 and 4 hours': 3.5,
    'Between 4 and 5 hours': 4.5,
    'More than 5 hours': 6
}


In [ ]:
#Code to Apply the Mapping:
df_clean['8. What is the average time you spend on social media every day?'] = \
    df_clean['8. What is the average time you spend on social media every day?'].map(time_mapping)


In [ ]:
# Check how many entries fall into each mapped category
print(df_clean['8. What is the average time you spend on social media every day?'].value_counts().sort_index())


8. What is the average time you spend on social media every day?
1     30
2     62
3     92
4    253
Name: count, dtype: int64


In [ ]:
df_clean.columns.tolist()

['1. What is your age?',
 '6. Do you use social media?',
 '8. What is the average time you spend on social media every day?',
 '9. How often do you find yourself using Social media without a specific purpose?',
 '10. How often do you get distracted by Social media when you are busy doing something?',
 "11. Do you feel restless if you haven't used Social media in a while?",
 '12. On a scale of 1 to 5, how easily distracted are you?',
 '13. On a scale of 1 to 5, how much are you bothered by worries?',
 '14. Do you find it difficult to concentrate on things?',
 '15. On a scale of 1-5, how often do you compare yourself to other successful people through the use of social media?',
 '16. Following the previous question, how do you feel about these comparisons, generally speaking?',
 '17. How often do you look to seek validation from features of social media?',
 '18. How often do you feel depressed or down?',
 '19. On a scale of 1 to 5, how frequently does your interest in daily activities fl

In [ ]:
df_clean['depression_binary'] = df_clean['18. How often do you feel depressed or down?'].apply(lambda x: 1 if x >= 3 else 0)

In [ ]:
# Save final cleaned and preprocessed dataset to CSV
df_clean.to_csv("smmh_final_clean.csv", index=False)

In [ ]:
# Assume df_clean is your cleaned and feature-engineered DataFrame
# and 'depression_binary' is your target variable

# Step 1: Reorder columns to move target to the end
other_columns = [col for col in df_clean.columns if col != 'depression_binary']
df_clean = df_clean[other_columns + ['depression_binary']]

# Step 2: Save to CSV
df_clean.to_csv("smmh_final_model_ready.csv", index=False)
print("Reordered and saved as 'smmh_final_model_ready.csv'")


Reordered and saved as 'smmh_final_model_ready.csv'
